# Load Libraries

In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Set Path

In [ ]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")

# Load Data

In [ ]:
df_train = pd.read_pickle(str(data_dir / "gsdc_cleaned_train.pkl.gzip"))

In [ ]:
df_test = pd.read_pickle(str(data_dir / "gsdc_cleaned_test.pkl.gzip"))

In [ ]:
for col in df_train.columns:
    print(col)

# Extract Data

## Get dlatDeg, dlngDeg, dheightAboveWgs84EllipsoidM

In [ ]:
def getDeltaPositions(df:pd.DataFrame)->pd.DataFrame:
    output = df.copy()
    phones = output['phone'].unique()
    output[['dlatDeg', 'dlngDeg', 'dheight']] = 0
    for phone in phones:
        latDeg = output.loc[output['phone'] == phone, 'latDeg']
        output.loc[output['phone'] == phone, 'dlatDeg'] = latDeg - latDeg.iloc[0]
        
        lngDeg = output.loc[output['phone'] == phone, 'lngDeg']
        output.loc[output['phone'] == phone, 'dlngDeg'] = lngDeg - lngDeg.iloc[0]
        
        heightAboveWgs84EllipsoidM = output.loc[output['phone'] == phone, 'heightAboveWgs84EllipsoidM']
        output.loc[output['phone'] == phone, 'dheight'] = heightAboveWgs84EllipsoidM - heightAboveWgs84EllipsoidM.iloc[0]
        
    return output

In [ ]:
df_train = getDeltaPositions(df_train)
df_test = getDeltaPositions(df_test)

## Scaler

In [ ]:
class Scaler():
    def __init__(self, target_columns):
        self.target_columns = target_columns
        self.renames_columns = [name + "_Scaled" for name in target_columns]
        self.scaler = dict()
        for name in target_columns:
            self.scaler[name] = StandardScaler()
            
    def fit(self, df:pd.DataFrame):
        for col in self.target_columns:
            self.scaler[col].fit(df[col].values.reshape(-1, 1))
            
    def transform(self, df:pd.DataFrame):
        output = df.copy()
        for col,recol in zip(self.target_columns, self.renames_columns):
            output[recol] = self.scaler[col].transform(df[col].values.reshape(-1, 1))
        return output
        
    

In [ ]:
target_columns = ['dlatDeg', 'dlngDeg', 'dheight',
                 'xSatPosM', 'ySatPosM', 'zSatPosM', 'xSatVelMps', 'ySatVelMps', 'zSatVelMps',
                  'UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec', 
                  'DriftXRadPerSec', 'DriftYRadPerSec', 'DriftZRadPerSec', 
                  'UncalAccelXMps2', 'UncalAccelYMps2', 'UncalAccelZMps2', 'BiasXMps2', 'BiasYMps2', 'BiasZMps2',
                  'UncalMagXMicroT', 'UncalMagYMicroT', 'UncalMagZMicroT', 'BiasXMicroT', 'BiasYMicroT', 'BiasZMicroT',
                  'yawDeg', 'rollDeg', 'pitchDeg'
                 ]

In [ ]:
scaler = Scaler(target_columns)
scaler.fit(df_train)

df_train = scaler.transform(df_train)
df_test = scaler.transform(df_test)

In [ ]:
df_train[scaler.renames_columns].describe()

## Categorical Variables
### signalType

In [ ]:
categories_signal_type = ['GPS_L1', 'GPS_L5', 'GAL_E1', 'GAL_E5A', 'GLO_G1', 'BDS_B1I', 'BDS_B1C', 'BDS_B2A', 'QZS_J1', 'QZS_J5']


In [ ]:
df_train['signalType'].value_counts()

In [ ]:
df_test['signalType'].value_counts()

In [ ]:
def GetSignalTypeDummies(df:pd.DataFrame, cols = ['GPS_L1', 'GPS_L5', 'GAL_E1', 'GAL_E5A', 'GLO_G1', 'BDS_B1I', 'BDS_B1C', 'BDS_B2A', 'QZS_J1', 'QZS_J5']):
    output = df.copy()
    for col in cols:
        output[col] = (df['signalType'] == col).astype(int)
    return output    

In [ ]:
df_train = GetSignalTypeDummies(df_train)
df_test = GetSignalTypeDummies(df_test)

In [ ]:
print(df_train.shape)
print(df_test.shape)

#  Output

In [ ]:
df_train.to_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))
df_test.to_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))